In [ ]:
#이 프로그램에서 Ensemble 테스트는 각 모델별 결과를 이어받는 형태로 기존 방식으로 구현
#신규방식인 각 모델 결과를 모아서 최적의 영역을 선택 추출하는 방법은 new_RetinaNet - ENSEMBLE_TEST.ipynb로 구현

import os
import pandas as pd
from itertools import groupby
from itertools import chain
from operator import eq
import keras
import keras.preprocessing.image
from keras_retinanet.models.resnet import custom_objects
# from keras_retinanet.preprocessing.coco import CocoGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
import tensorflow as tf

%matplotlib inline
%load_ext autoreload
%autoreload 2

OS_ENV = 'WIN' # 'LINUX' or 'WIN'
MODEL_ENV = 'separate' # 'ensemble' or 'separate'
DATE = '20181031'
DATA_TO_TEST = 'abn_all' # 'abn1' or 'abn2' or 'abn_all' or 'normal' or 'normal_1'
#normal_1': #2018. 1월말 데이터중 Normal 1318 file

#SCORE_VALUE_LIST = [0.5,0.4]
SCORE_VALUE_LIST = [0.4]

In [ ]:
if OS_ENV == 'LINUX':
    result_save_base_path = '/home/huray/workspace/han_work/han_test/results/'
else:
    result_save_base_path = 'D:/Work/NCC/han_test/results/'
result_save_subpath = os.path.join(result_save_base_path, DATE + '_' + MODEL_ENV, DATA_TO_TEST)
result_save_path = result_save_subpath
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

########## 2D list에서 특정 값이 위치한 index return ####################
def find(searchList, elem):
    for ix, row in enumerate(searchList):
        for iy, i in enumerate(row):
            if i==elem:
                #print('{},{}'.format(ix,iy))
                return [ix,iy]
    return []
        
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
keras.backend.tensorflow_backend.set_session(get_session())

## Load RetinaNet model

In [ ]:
model_list = []
#resnet50, from resnet50_ep90, le-7
#model_list.append('snapshots/resnet50_csv_01.h5') #31
#model_list.append('snapshots/resnet50_csv_02.h5') #60
'''
model_list.append('snapshots/resnet50_csv_03.h5') #
model_list.append('snapshots/resnet50_csv_04.h5') #
model_list.append('snapshots/resnet50_csv_05.h5') #
model_list.append('snapshots/resnet50_csv_06.h5') #
model_list.append('snapshots/resnet50_csv_07.h5') #
model_list.append('snapshots/resnet50_csv_08.h5') #
model_list.append('snapshots/resnet50_csv_09.h5') #
model_list.append('snapshots/resnet50_csv_10.h5') #
model_list.append('snapshots/resnet50_csv_11.h5') #
model_list.append('snapshots/resnet50_csv_12.h5') #
model_list.append('snapshots/resnet50_csv_13.h5') #
model_list.append('snapshots/resnet50_csv_14.h5') #
model_list.append('snapshots/resnet50_csv_15.h5') #
model_list.append('snapshots/resnet50_csv_16.h5') #
model_list.append('snapshots/resnet50_csv_17.h5') #
model_list.append('snapshots/resnet50_csv_18.h5') #
model_list.append('snapshots/resnet50_csv_19.h5') #
model_list.append('snapshots/resnet50_csv_20.h5') #
model_list.append('snapshots/resnet50_csv_21.h5') #
model_list.append('snapshots/resnet50_csv_22.h5') #
model_list.append('snapshots/resnet50_csv_23.h5') #
model_list.append('snapshots/resnet50_csv_24.h5') #
model_list.append('snapshots/resnet50_csv_25.h5') #
model_list.append('snapshots/resnet50_csv_26.h5') #
model_list.append('snapshots/resnet50_csv_27.h5') #
model_list.append('snapshots/resnet50_csv_28.h5') #
model_list.append('snapshots/resnet50_csv_29.h5') #
model_list.append('snapshots/resnet50_csv_30.h5') #
model_list.append('snapshots/resnet50_csv_31.h5') #
model_list.append('snapshots/resnet50_csv_32.h5') #
model_list.append('snapshots/resnet50_csv_33.h5') #
model_list.append('snapshots/resnet50_csv_34.h5') #
model_list.append('snapshots/resnet50_csv_35.h5') #
model_list.append('snapshots/resnet50_csv_36.h5') #
model_list.append('snapshots/resnet50_csv_37.h5') #
model_list.append('snapshots/resnet50_csv_38.h5') #
model_list.append('snapshots/resnet50_csv_39.h5') #
model_list.append('snapshots/resnet50_csv_40.h5') #
model_list.append('snapshots/resnet50_csv_41.h5') #
model_list.append('snapshots/resnet50_csv_42.h5') #
model_list.append('snapshots/resnet50_csv_43.h5') #
model_list.append('snapshots/resnet50_csv_44.h5') #
model_list.append('snapshots/resnet50_csv_45.h5') #
model_list.append('snapshots/resnet50_csv_46.h5') #
model_list.append('snapshots/resnet50_csv_47.h5') #
model_list.append('snapshots/resnet50_csv_48.h5') #
model_list.append('snapshots/resnet50_csv_49.h5') #
model_list.append('snapshots/resnet50_csv_50.h5') #
model_list.append('snapshots/resnet50_csv_51.h5') #
model_list.append('snapshots/resnet50_csv_52.h5') #
model_list.append('snapshots/resnet50_csv_53.h5') #
model_list.append('snapshots/resnet50_csv_54.h5') #
model_list.append('snapshots/resnet50_csv_55.h5') #
model_list.append('snapshots/resnet50_csv_56.h5') #
model_list.append('snapshots/resnet50_csv_57.h5') #
model_list.append('snapshots/resnet50_csv_58.h5') #
model_list.append('snapshots/resnet50_csv_59.h5') #
model_list.append('snapshots/resnet50_csv_60.h5') #
model_list.append('snapshots/resnet50_csv_61.h5') #
model_list.append('snapshots/resnet50_csv_62.h5') #
model_list.append('snapshots/resnet50_csv_63.h5') #
model_list.append('snapshots/resnet50_csv_64.h5') #
model_list.append('snapshots/resnet50_csv_65.h5') #
model_list.append('snapshots/resnet50_csv_66.h5') #
model_list.append('snapshots/resnet50_csv_67.h5') #
model_list.append('snapshots/resnet50_csv_68.h5') #
model_list.append('snapshots/resnet50_csv_69.h5') #
model_list.append('snapshots/resnet50_csv_70.h5') #
model_list.append('snapshots/resnet50_csv_71.h5') #
model_list.append('snapshots/resnet50_csv_72.h5') #
model_list.append('snapshots/resnet50_csv_73.h5') #
model_list.append('snapshots/resnet50_csv_74.h5') #
model_list.append('snapshots/resnet50_csv_75.h5') #
model_list.append('snapshots/resnet50_csv_76.h5') #
model_list.append('snapshots/resnet50_csv_77.h5') #
model_list.append('snapshots/resnet50_csv_78.h5') #
model_list.append('snapshots/resnet50_csv_79.h5') #
model_list.append('snapshots/resnet50_csv_80.h5') #
model_list.append('snapshots/resnet50_csv_81.h5') #
model_list.append('snapshots/resnet50_csv_82.h5') #
model_list.append('snapshots/resnet50_csv_83.h5') #
model_list.append('snapshots/resnet50_csv_84.h5') #
model_list.append('snapshots/resnet50_csv_85.h5') #
model_list.append('snapshots/resnet50_csv_86.h5') #
model_list.append('snapshots/resnet50_csv_87.h5') #
model_list.append('snapshots/resnet50_csv_88.h5') #
model_list.append('snapshots/resnet50_csv_89.h5') #
model_list.append('snapshots/resnet50_csv_90.h5') # 
'''

#---------------------------------resnet50---------------------------------------
#1 : 1e-5
#model_list.append('snapshots/resnet50/resnet50_csv_84.h5')
#2 : from ep90, 1e-5, 4376
#model_list.append('snapshots/resnet50/from_e90/resnet50_csv_03.h5')
#model_list.append('snapshots/resnet50/from_e90/resnet50_csv_19.h5')
#model_list.append('snapshots/resnet50/from_e90/resnet50_csv_27.h5')
#model_list.append('snapshots/resnet50/from_e90/resnet50_csv_36.h5')
#3 : from ep90, 1e-6, 4617(이전 비정상 테스트 데이터 추가 학습)
#model_list.append('snapshots/resnet50/from_e90_4617/resnet50_csv_49.h5')
#model_list.append('snapshots/resnet50/from_e90_4617/resnet50_csv_65.h5')
#model_list.append('snapshots/resnet50/from_e90_4617/resnet50_csv_77.h5')
#model_list.append('snapshots/resnet50/from_e90_4617/resnet50_csv_80.h5')


#---------------------------------resnet101---------------------------------------
#1 : 1e-5
#model_list.append('snapshots/resnet101/resnet101_csv_61.h5')
###model_list.append('snapshots/resnet101/resnet101_csv_59.h5')
#2 : from ep61, 1e-5, 4376
#model_list.append('snapshots/resnet101/from_e61/resnet101_csv_35.h5')
#model_list.append('snapshots/resnet101/from_e61/resnet101_csv_29.h5')
#model_list.append('snapshots/resnet101/from_e61/resnet101_csv_56.h5')
#3 : from ep59, 1e-7, 4617(이전 비정상 테스트 데이터 추가 학습)
#model_list.append('snapshots/resnet101/from_e59_4617/resnet101_csv_77.h5')
#model_list.append('snapshots/resnet101/from_e59_4617/resnet101_csv_82.h5')
#model_list.append('snapshots/resnet101/from_e59_4617/resnet101_csv_90.h5')
#model_list.append('snapshots/resnet101/from_e59_4617/resnet101_csv_91.h5')


#---------------------------------resnet152---------------------------------------
#1 : 1e-5
#model_list.append('snapshots/resnet152/resnet152_csv_74.h5')
#2 : from ep74, 1e-5, 4376
#model_list.append('snapshots/resnet152/from_e74/resnet152_csv_32.h5')
#model_list.append('snapshots/resnet152/from_e74/resnet152_csv_45.h5')
#model_list.append('snapshots/resnet152/from_e74/resnet152_csv_53.h5')
#model_list.append('snapshots/resnet152/from_e74/resnet152_csv_57.h5')
#model_list.append('snapshots/resnet152/from_e74/resnet152_csv_84.h5')
#3 : from ep74, 1e-7, 4617(이전 비정상 테스트 데이터 추가 학습)
#model_list.append('snapshots/resnet152/from_e74_4617/resnet152_csv_53.h5')
#model_list.append('snapshots/resnet152/from_e74_4617/resnet152_csv_55.h5')
#model_list.append('snapshots/resnet152/from_e74_4617/resnet152_csv_57.h5')
#model_list.append('snapshots/resnet152/from_e74_4617/resnet152_csv_71.h5')



####################################### 이전모델 ###################################
#ensemble 아래 e44 + e29
#single e29
#model_list.append('snapshots/0326_more_aug/resnet101_csv_44.h5')   
#model_list.append('snapshots/0525_add_ncc_mass_data_strong_aug/resnet101_csv_29.h5')
#######

#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_25.h5')
#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_29.h5')
#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_47.h5')
#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_48.h5')
#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_49.h5')

### Initialize data generators

In [ ]:
# create image data generator object
val_image_data_generator = keras.preprocessing.image.ImageDataGenerator()

if DATA_TO_TEST == 'abn1':
    if OS_ENV == 'LINUX':
        data_csv_path = '/home/huray/data/NCC/img_retinanet/data_abn1.csv'
    else:
        data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/data_abn1_win.csv'
elif DATA_TO_TEST == 'abn2':
    if OS_ENV == 'LINUX':
        data_csv_path = '/home/huray/data/NCC/img_retinanet/data_abn2.csv'
    else:
        data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/data_abn2_win.csv'
elif DATA_TO_TEST == 'normal':
    if OS_ENV == 'LINUX':    
        data_csv_path = '/home/huray/data/NCC/img_retinanet/data_normal.csv'
    else:
        data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/data_normal_win.csv'
elif DATA_TO_TEST == 'normal_1': #2018. 1월말 데이터중 Normal 1381 file
    if OS_ENV == 'LINUX':    
        data_csv_path = '/home/huray/data/NCC_trainset/img_retinanet/data_ncc_normal_180130.csv'
    else:
        data_csv_path = 'D:/Work/NCC/data/NCC_trainset/img_retinanet/data_ncc_normal_180130.csv'
elif DATA_TO_TEST == 'abn_all':
    if OS_ENV == 'LINUX':
        data_csv_path = '/home/huray/data/NCC/img_retinanet/data_abn_merged.csv'
        #data_csv_path = '/home/huray/data/NCC/img_retinanet/0914_new_NCC_test_only_abn.csv'
        #data_csv_path = '/home/huray/data/NCC/img_retinanet/0914_new_NCC_test_only_abn_imsi.csv'
    else:
        data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/data_abn_merged_win.csv'        
        #data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/0914_new_NCC_test_only_abn_anno_for_win.csv'
        #data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/0914_new_NCC_test_only_abn_for_win.csv' 
else:
    print('WRONG DATA CSV PATH!')
    raise
    
if OS_ENV == 'LINUX':
    label_csv_path = '/home/huray/data/new_trainset/class_2_malig_or_norm.csv'
else:
    label_csv_path = 'D:/Work/NCC/data/new_trainset/class_2_malig_or_norm.csv'

# create a generator for testing data
val_generator = CSVGenerator(
    data_csv_path,
    label_csv_path,
    transform_generator=val_image_data_generator,
    batch_size=1
)

test_file_df = pd.read_csv(data_csv_path, header=None)
num_of_test_files = len(test_file_df)
print(num_of_test_files)

test_file_list = list(test_file_df[0]) #csv에서 읽어온 테스트 이미지파일 정보 목록
test_file_list.sort(key=lambda x: '-'.join(x.split('-')[:2])) #테스트 파일 목록을 groupby 하기 위해서 정렬

## define and initialize

In [ ]:
number_abnormal_detect_from_normal = 0 #normal 파일에서 abnormal로 검출된 경우의 수
#img_index_list = list(range(0, num_of_test_files))
not_detected_img_index_list = list(range(0, num_of_test_files))
#print(len(not_detected_img_index_list))

ordinal=lambda x:["1st","2nd","3rd","4th","5th","6th"][x-1]

############# for 테스트 파일목록 Groupping ############
groupby_key_list = [] #그룹명(테스트 파일 단위 그룹 명) 목록
groupby_file_list = [] #그룹화된 테스트파일 목록
groupby_detected_number_for_files = [] #검출(detect)된 수에 대한 그룹화된 목록(그룹화된 테스트 목록 구조와 동일하게 매핑)
#list_not_detected_group_index = [] #검출되지 않은 그룹의 index 목록 

#M_max_scores = [] #abnormal 테스트 파일에서 detect된 anchor score 중 가장 높은 score
#N_max_scores = [] #normal 각 테스트 파일에서 detect된 anchor score 중 가장 높은 score

#테스트 파일 목록을 group by하여 list에 저장
#groupby key(그룹핑 기준(단위)는 파일명을 '-'로 split 한뒤 앞쪽 2번째 요소까지 추출함)
for key, group in groupby(test_file_list, key=lambda x: '-'.join(x.split('-')[:2])):
    #그룹 명(key) 목록
    groupby_key_list.append(key) 
    temp = list(group)
    #그룹화된 전체 테스트파일 목록 
    groupby_file_list.append(temp) 
    #그룹화 된 테스트 파일별 검출정보 목록(그룹화된 전체 테스트파일 목록과 동일한 구조로 구성),0 으로 초기화
    groupby_detected_number_for_files.append([0]*len(temp))

    #그룹화 된 테스트 파일별 검출된 최대 score정보 목록(그룹화된 전체 테스트파일 목록과 동일한 구조로 구성),0 으로 초기화
    #M_max_scores.append([0]*len(temp))
    
    
    
############################# !!!!!!!!!!!!!!!!!!!!!!!!!!! #############################
###테스트 결과 출력
def get_detection_info():
    list_not_detected_group_index = []  # 검출되지 않은 그룹의 index 목록
    total_not_detected_file = 0 #검출되지 않은 전체 테스트 파일의 수    
    i_group_member = 0

    print('------------------------------------------------ Result info --------------------------------------------------------')
    #print('\n')    
    
    for idx_x, row in enumerate(groupby_detected_number_for_files):
        i_detections = 0
        for idx_y, elem in enumerate(row):
            i_detections += elem
            if elem == 0:
                total_not_detected_file += 1
                
        if i_detections == 0:
            list_not_detected_group_index.append(idx_x)
            i_group_member += len(row)
            print('({})set have no detection!.'.format(groupby_key_list[idx_x]))

    #print('\n')

    #1st print type
    #print('Total Number of abnormal case not detected :::::::::: {}'.format(total_not_detected_file))
    #print('Number of abnormal case set not detected::::::: {}({} files)'.format(len(list_not_detected_group_index), i_group_member))

    #2nd print type
    #print('{}/{} files not detected.'.format(total_not_detected_file,len(test_file_list)))
    #print('{}({} files)/{} sets not detected.'.format(len(list_not_detected_group_index), i_group_member, len(groupby_key_list)))

    #3rd print type
    print('{} files'.format(total_not_detected_file))
    print('{}({} files) sets  not detected.'.format(len(list_not_detected_group_index), i_group_member))
    #print('\n')
    
    if 'normal' in DATA_TO_TEST:
        print('{} abnormal detected.(from {} normal files).'.format(number_abnormal_detect_from_normal,len(test_file_list)))
    
    print('---------------------------------------------------------------------------------------------------------------------')
    
def init_info_list():
    for idx_i, row in enumerate(groupby_file_list):
        for idx_j, col in enumerate(row):
            groupby_detected_number_for_files[idx_i][idx_j] = 0
            #M_max_scores[idx_i][idx_j] = 0


## Run detection

In [ ]:
for SCORE_VALUE in SCORE_VALUE_LIST:
    print('****************{}*******************'.format(MODEL_ENV))
    print('*      Detecting with score {}           *'.format(SCORE_VALUE))
    print('*******************************************')
    
    model_count = 0;
    for model_path in model_list:
        #ensemble 테스트가 아닐경우 각 model별 result folder를 만들어 결과(file별 detection 표시)를 저장
        #ensemble일 경우는 ensemble folder에 최종 결과를 저장
        if not MODEL_ENV == 'ensemble': #ensemble 테스트가 아닐경우        
            result_save_path = os.path.join(result_save_subpath, 'score'+str(SCORE_VALUE), model_path.split('/')[-1].split('.')[-2])
            if not os.path.exists(result_save_path):
                os.makedirs(result_save_path)        

        model_count += 1
        number_abnormal_detect_from_normal = 0 #normal 파일에서 abnormal로 검출된 경우의 수
        number_of_proper_detection = 0 #검출된 수

        if MODEL_ENV == 'ensemble': #ensemble 테스트        
            print('\n{} model {}'.format(ordinal(model_count), model_path.split('/')[-1]))
        else:    
            print('\n{}. Load model {}'.format(model_count, model_path.split('/')[-1]))
        
        # Load RetinaNet Model
        model = keras.models.load_model(model_path, custom_objects=custom_objects)

        img_index_list = not_detected_img_index_list.copy() #검출이 안된 파일 목록, 처음에는 전체 파일 목록이 대상이 됨.
        num_of_not_detected_files = len(img_index_list)
        print('{} test files.'.format(num_of_not_detected_files))
        
        print("start.")
        start = time.time()
        #print('0/{} completed.\r'.format(num_of_test_files), end='')
        #for index in img_index_list:
        for index_count, index in enumerate(img_index_list):            
            pass_this_file = False

            # load image
            image, image_path = val_generator.load_image(index, get_image_path=True)
            #print('image :::::::::: {}'.format(image_path))
            image_name_split = image_path.split('/')[-1].split('-')

            # copy to draw on
            draw = image.copy()
            draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

            # preprocess image for network
            image = val_generator.preprocess_image(image)
            image, scale = val_generator.resize_image(image)
            #annotations = val_generator.load_annotations(index)


            # process image
            _, _, detections = model.predict_on_batch(np.expand_dims(image, axis=0))

            # compute predicted labels and scores
            predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
            scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]

            # correct for image scale
            detections[0, :, :4] /= scale

            number_of_proper_detection = 0 #현재 파일에서 detect된 수
            '''
            M_max_score = 0 
            N_max_score = 0
            '''
            for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
                '''
                if label == 0:
                    if M_max_score < score:
                        M_max_score = score    
                else:
                    if N_max_score < score:
                        N_max_score = score
                '''

                if score < SCORE_VALUE or label != 0:
                    continue

                number_of_proper_detection += 1
                
                #for detect된 정보 visualization
                b = detections[0, idx, :4].astype(int)
                
                #cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 5)
                cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 2)
                caption = "{} {:.3f}".format(val_generator.label_to_name(label), score)
                cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 5)
                cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)
                
            '''
            # 테스트 파일 그룹 목록에서 현재 파일의 위치를 구함    
            img_indics = find(groupby_file_list, image_path)
            if len(img_indics) > 0:                
                # 검출된 정보 중 Max 스코어 값을 파일 위치와 매칭하여 저장
                M_max_scores[img_indics[0]][img_indics[1]] = M_max_score 
            '''

            #이미지에서 (조건에 부합되게)detect된 것이 있을 경우
            if number_of_proper_detection > 0:
                if 'normal' in image_path:
                    number_abnormal_detect_from_normal += 1            

                ##################### 그룹화된 테스트 파일 정보와 동일한 구조의 detection 정보 목록에 반영 ##############
                # 테스트 파일 그룹 목록에서 현재 파일의 위치를 구함    
                img_indics = find(groupby_file_list, image_path)
                if len(img_indics) > 0:
                    #groupby_detected_number_for_files[img_indics[0]][img_indics[1]] += number_of_proper_detection
                    groupby_detected_number_for_files[img_indics[0]][img_indics[1]] = 1
                    
                #Ensemble일경우 테스트 파일 목록에서 검출된 파일 목록을 삭제(다음 모델 테스트에서 제외)
                if MODEL_ENV == 'ensemble': #ensemble 테스트일 경우                
                    not_detected_img_index_list.remove(index) 

                # save the image into file
                #cv2.imwrite(os.path.join(result_save_path, '-'.join(image_path.split('/')[-2:])), draw)
                #print(os.path.join(result_save_path, '-'.join(image_path.split('/')[-2:])))
                    
            #print('{}/{} completed.\r'.format(index_count+1,num_of_test_files), end='')
            print('{}/{} completed.\r'.format(index_count+1,num_of_not_detected_files), end='')
            
        print("end({:.2f} minutes elapsed).".format((time.time() - start)/60))
        
        if not MODEL_ENV == 'ensemble': #ensemble 테스트가 아닐경우
            get_detection_info() #detection 결과 출력(각 모델별로 출력)
            init_info_list() #next model로 변경을 위해 현재 model의 detection 정보를 초기화

        # release the memory
        del model
        keras.backend.clear_session()

    if MODEL_ENV == 'ensemble': #ensemble 테스트일 경우
        get_detection_info() #detection 결과 출력(테스트를 모두 끝내고 출력)
        
    print('\n')
    
    init_info_list() #next score로 변경을 위해 detection 정보를 초기화

In [ ]:
print('Finished!')